In [104]:
n = 4
N=binomial(n,2)
#N=4

In [105]:
#list of unordered pairs of elements in {1,...,n}
def pairs(n): return flatten([[[i+1,j+1] for j in range(i+1,n)] for i in range(n-1)],max_level=1)

#sigma acts on an unordered pair
def act(sigma, p): return sorted([sigma(p[0]),sigma(p[1])])

#sigma acts as matrix in basis on a vector v
def act_vect(g,v,basis=identity_matrix(N)): return (basis.inverse()*g.matrix()*basis)*vector(v)

#permutation in Sigma_N which induced from perm in Sigma_n
def embed(sigma,n): return perm_from_sort([act(sigma,p) for p in pairs(n)])

#find permutation which orders a list L	
def perm_from_sort(L): return Permutation([pair[0] for pair in sorted(enumerate(L, 1), key=lambda x: x[1])]).to_cycles()

#image of generators of Sigma_n under embedding into Sigma_N
def embed_gens(n): return [embed(gen,n) for gen in SymmetricGroup(n).gens()]

In [106]:
#fundamental domain given arbitrary distinct vector l and basis B
def fund_domain(center=[i for i in range(N)],basis=identity_matrix(N),br=QQ,group=MatrixGroup(identity_matrix(N))):

    #augmented matrix for half-plane ineqs
    A = [ [center[j] - act_vect(group[i],vector(center),basis)[j] for j in range(N)] for i in range(group.order())]
    b_1 = group.order()*[0]
    bA = matrix(br,b_1).transpose().augment(matrix(br,A))

    #augmented matrix for positivity ineqs
    pos = basis.inverse().transpose()
    b_2 = [0 for i in range(N)]
    bPos = matrix(br,b_2).transpose().augment(pos)

    #augmented matrix for fund. domain
    aug_l = bA.stack(bPos)
    
    #find convex polyhedral region, i.e. intersection of all ineqs
    poly = Polyhedron(ieqs=aug_l,base_ring=br)

    return([aug_l,poly])

In [107]:
#slice the fundamental domain with the plane x_1 + ... + x_N = 1
def cross_section(region,slice_dir=[1 for i in range(N)],chi=1,br=None):
    if br is None: br=region.base_ring()
    return Polyhedron(ieqs=region.inequalities(),eqns=[[-1]+slice_dir],base_ring=br)

In [108]:
#find fixed points under G in polyhedron F to include as new vertices
#find intersection of fixed point subspace with F as polyhedron, and return those vertices
def fixed_verts(F,br=QQ):
    new_verts=set()
    for g in G:
        #get equations for fixed point subspace
        A=g.matrix() #matrix associated to group element g
        B=A-identity_matrix(N) #eqns defining fixed pt subspace are (A-Id)x == 0
        b = N*[0] #0 vector
        bB = matrix(br,b).transpose().augment(matrix(br,B))#form augmented matrix for equations defining subspace
        #get equations for intersection
        eqns=matrix(br,[list(eq) for eq in F.equations()]) #list of equations defining F
        eqns=eqns.stack(bB) #augmented matrix including fixed point subspace equations
        #get list of inequalities for fundamental domain as convex polyhedron
        ieqs=matrix(br,[list(ieq) for ieq in F.inequalities()])
        #form intersection of fund_domain with fixed point subspace
        intersection_F_fixed_pt_subspace=Polyhedron(ieqs=ieqs,eqns=eqns)
        verts=intersection_F_fixed_pt_subspace.vertices()
        for vert in verts:
            new_verts.add(tuple(vert))
    return sorted(list(new_verts))

In [109]:
#action of group element g on a face
#if g.face is not in fund_domain defined by vertex list, return None
def act_face(g,face,vertices):
    try:
        return [vertices.index(tuple(act_vect(g,vertices[i]))) for i in face]
    except ValueError:
        return None

In [110]:
#determine if face1 is glued to face2 by the action of G
#faces are given by a list vertex indices
def faces_glued(face1,face2,G,vertices):
    for g in G:
        g_face1 = act_face(g,face1,vertices)
        if g_face1 is not None and set(g_face1) == set(face2):
            return True
    return False

In [111]:
#for two glued faces face1, face2, determine if the gluing map preserves orientation
#if a gluing map g1 preserves orientation, then all do since g1*g2^(-1) is a self map and must be trivial
def gluing_preserves_orientation(face1,face2,vertices):
    S=SymmetricGroup(range(len(vertices))) #symmetric group on vertex indices
    for g in G:
        g_face1 = act_face(g,face1,vertices)
        if g_face1 is not None and set(g_face1) == set(face2):
            sigma_g=perm_from_sort(g_face1)
            g_cycle=PermutationGroupElement(sigma_g,parent=S) #get permutation which orders g.v_{i_0}...g.v_{i_k}
            return g_cycle.sign()
    return None

In [112]:
#determine if there is group element which glues face non-trivially to itself
def trivial_self_gluing(face,G,vertices):
    trivial = True
    for g in G:
        #act on vertices defining face
        g_face = act_face(g,face,vertices)
        if g_face is not None and set(g_face) == set(face):
            #check if the gluing is non-trivial
            if [g_face.index(i) for i in face] != [i for i in range(len(face))]:
                trivial = False
    return trivial

In [113]:
#toss out any facets with non-trivial self-gluings such as [0,3,4] or [1,6]
#build list of faces for each dim
import itertools
def faces(k): return [face for face in itertools.combinations(range(len(vertices)), k+1) if trivial_self_gluing(face,G,vertices)]

In [114]:
#find classes of vertices which are glued
def glued_faces(k):
    glued_verts = []
    for i in faces(k):
        found = False
        for equiv_class in glued_verts:
            if len(equiv_class) >= 1:
                if faces_glued(i,equiv_class[0],G,vertices):
                    equiv_class.append(i)
                    found=True
        if not found:
            glued_verts.append([i])
    return glued_verts

In [115]:
#compute boundary of each face. keep track of orientation
def boundary(face,glued_faces,vertices):
    dim = len(face)-1 #face/simplex dimension is number of vertices in face-1
    boundary = [0]*len(glued_faces[dim-1]) #initialize vector to count occurence boundary faces
    for i in range(len(face)):
        face_remove_i = face[:i] + face[i+1:] #remove vertex at index i
        sign = (-1)^i
        #find representative to which face\{i} is glued to
        for glued_face in glued_faces[dim-1]:
            face_rep = glued_face[0] #use first face in glued_face list as representative
            face_index = None
            if face_remove_i in glued_face: #check if face_remove_i is in gluing class
                face_index = glued_faces[dim-1].index(glued_face) #get index of face in glued (k-1)-faces
                orient_preserve = gluing_preserves_orientation(face_rep,face_remove_i,vertices) #determine if gluing map preserves orientation
            if face_index is not None:
                boundary[face_index] += orient_preserve*sign #add up using index
    return boundary

In [116]:
#compute the boundary map matrix over \ZZ
def boundary_map(k): return matrix(ZZ,[boundary(face[0],glued_face_list,vertices) for face in glued_face_list[k]]).transpose()

In [117]:
#define a chain complex, optionally truncated at max_degree
def chain_complex(max_degree=1): return ChainComplex({k:boundary_map(k) for k in range(1,max_degree+1)},degree=-1)

In [118]:
#define the finite group
G = PermutationGroup(embed_gens(n)) #symmetric group \sigma_n as a subgroup of \sigma_N, with 2 generators
#G = MatrixGroup(matrix(QQ,[[0,0,0,1],[1,0,0,0],[0,1,0,0],[0,0,1,0]]))
#G=SymmetricGroup(4)

In [131]:
#compute fundamental domain to get list of vertices
F=fund_domain(group=G,center=[1,2,3,5,6,8]); F[1]

A 6-dimensional polyhedron in QQ^6 defined as the convex hull of 1 vertex and 16 rays (use the .plot() method to plot)

In [132]:
#compute a cross section of the fundamental domain (sum of components is constant)
F2=cross_section(F[1])
[tuple(v) for v in F2.vertices()]

[(0, 0, 0, 1/3, 1/3, 1/3),
 (0, 0, 1/3, 0, 1/3, 1/3),
 (0, 0, 9/22, 25/66, 7/66, 7/66),
 (25/66, 0, 7/66, 0, 7/66, 9/22),
 (2/9, 0, 2/9, 2/9, 1/9, 2/9),
 (9/22, 0, 0, 7/66, 7/66, 25/66),
 (1/4, 0, 1/4, 1/4, 0, 1/4),
 (1/6, 1/6, 1/6, 1/6, 1/6, 1/6),
 (1/2, 0, 0, 0, 0, 1/2),
 (1/4, 1/4, 0, 0, 1/4, 1/4),
 (2/5, 0, 0, 0, 1/5, 2/5),
 (0, 0, 0, 0, 1/2, 1/2),
 (0, 2/5, 0, 0, 2/5, 1/5),
 (0, 0, 0, 0, 0, 1),
 (0, 2/9, 2/9, 2/9, 2/9, 1/9),
 (0, 0, 2/5, 2/5, 0, 1/5)]

In [133]:
vertices=fixed_verts(F2,QQ); vertices

[(0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 1/2, 1/2),
 (0, 0, 0, 1/3, 1/3, 1/3),
 (0, 0, 1/3, 0, 1/3, 1/3),
 (0, 0, 2/5, 2/5, 0, 1/5),
 (0, 0, 9/22, 25/66, 7/66, 7/66),
 (0, 2/9, 2/9, 2/9, 2/9, 1/9),
 (0, 2/5, 0, 0, 2/5, 1/5),
 (1/6, 1/6, 1/6, 1/6, 1/6, 1/6),
 (2/9, 0, 2/9, 2/9, 1/9, 2/9),
 (1/4, 0, 1/4, 1/4, 0, 1/4),
 (1/4, 1/4, 0, 0, 1/4, 1/4),
 (25/66, 0, 7/66, 0, 7/66, 9/22),
 (2/5, 0, 0, 0, 1/5, 2/5),
 (9/22, 0, 0, 7/66, 7/66, 25/66),
 (1/2, 0, 0, 0, 0, 1/2)]

In [134]:
#compute glued 0-faces
zero_faces=glued_faces(0); zero_faces

[[(0,)],
 [(1,)],
 [(2,)],
 [(3,)],
 [(4,), (7,), (13,)],
 [(5,), (12,), (14,)],
 [(6,), (9,)],
 [(8,)],
 [(10,), (11,)],
 [(15,)]]

In [135]:
#compute glued 1-faces
one_faces=glued_faces(1); one_faces

[[(0, 1)],
 [(0, 2)],
 [(0, 3)],
 [(0, 4), (0, 7)],
 [(0, 5)],
 [(0, 6)],
 [(0, 8)],
 [(0, 9)],
 [(0, 10), (0, 11)],
 [(0, 12)],
 [(0, 13)],
 [(0, 14)],
 [(0, 15)],
 [(1, 2)],
 [(1, 3)],
 [(1, 4)],
 [(1, 5)],
 [(1, 6), (1, 9)],
 [(1, 7), (1, 13)],
 [(1, 8)],
 [(1, 10)],
 [(1, 11)],
 [(1, 12)],
 [(1, 14)],
 [(1, 15)],
 [(2, 3)],
 [(2, 4), (2, 7), (2, 13)],
 [(2, 5), (2, 14)],
 [(2, 6), (2, 9)],
 [(2, 8)],
 [(2, 10), (2, 11)],
 [(2, 12)],
 [(2, 15)],
 [(3, 4), (3, 7), (3, 13)],
 [(3, 5), (3, 12)],
 [(3, 6), (3, 9)],
 [(3, 8)],
 [(3, 10), (3, 11)],
 [(3, 14)],
 [(3, 15)],
 [(4, 5), (12, 13), (13, 14)],
 [(4, 6), (6, 7), (9, 13)],
 [(4, 8), (7, 8), (8, 13)],
 [(4, 9)],
 [(4, 10), (7, 11), (11, 13)],
 [(4, 11), (7, 10)],
 [(4, 12), (7, 12)],
 [(4, 13)],
 [(4, 14), (7, 14)],
 [(4, 15), (7, 15)],
 [(5, 6), (5, 9), (9, 12), (9, 14)],
 [(5, 7), (5, 13)],
 [(5, 8), (8, 12), (8, 14)],
 [(5, 10), (10, 12), (10, 14), (11, 12), (11, 14)],
 [(5, 11)],
 [(5, 15)],
 [(6, 8), (8, 9)],
 [(6, 10), (6, 11)

In [ ]:
two_faces=glued_faces(2); two_faces

In [91]:
three_faces=glued_faces(3); three_faces

[[(0, 1, 2, 3)],
 [(0, 1, 2, 4)],
 [(0, 1, 2, 5)],
 [(0, 1, 2, 6)],
 [(0, 1, 2, 7)],
 [(0, 1, 2, 8)],
 [(0, 1, 2, 9)],
 [(0, 1, 2, 10)],
 [(0, 1, 3, 4)],
 [(0, 1, 3, 5)],
 [(0, 1, 3, 6)],
 [(0, 1, 3, 7)],
 [(0, 1, 3, 8)],
 [(0, 1, 3, 9)],
 [(0, 1, 3, 10)],
 [(0, 1, 4, 5)],
 [(0, 1, 4, 6)],
 [(0, 1, 4, 7)],
 [(0, 1, 4, 8)],
 [(0, 1, 4, 9)],
 [(0, 1, 4, 10)],
 [(0, 1, 5, 6)],
 [(0, 1, 5, 7)],
 [(0, 1, 5, 8)],
 [(0, 1, 5, 9)],
 [(0, 1, 5, 10)],
 [(0, 1, 6, 7)],
 [(0, 1, 6, 8)],
 [(0, 1, 6, 9)],
 [(0, 1, 6, 10)],
 [(0, 1, 7, 8)],
 [(0, 1, 7, 9)],
 [(0, 1, 7, 10)],
 [(0, 1, 8, 9)],
 [(0, 1, 8, 10)],
 [(0, 1, 9, 10)],
 [(0, 2, 3, 4)],
 [(0, 2, 3, 5)],
 [(0, 2, 3, 6)],
 [(0, 2, 3, 7)],
 [(0, 2, 3, 8)],
 [(0, 2, 3, 9)],
 [(0, 2, 3, 10)],
 [(0, 2, 4, 5), (0, 2, 5, 6)],
 [(0, 2, 4, 7), (0, 2, 6, 7)],
 [(0, 2, 4, 8), (0, 2, 6, 9)],
 [(0, 2, 4, 9), (0, 2, 6, 8)],
 [(0, 2, 4, 10), (0, 2, 6, 10)],
 [(0, 2, 5, 7)],
 [(0, 2, 5, 8), (0, 2, 5, 9)],
 [(0, 2, 5, 10)],
 [(0, 2, 7, 8), (0, 2, 7, 9)],
 [(0, 

In [ ]:
glued_face_list=[zero_faces,one_faces,two_faces]

In [ ]:
d1=boundary_map(1); d1

In [ ]:
d2=boundary_map(2); d2

In [ ]:
d3=boundary_map(3); d3

In [ ]:
cc = chain_complex(max_degree=2)

In [ ]:
cc.homology()

In [ ]:
list(map(lambda x: len(x), glued_face_list))